# Projeto 1 - Ciência dos Dados

Nome: Isabella Amorim 

Nome: Lucas Quadros

Nome: Nicolas Yassuda

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [27]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from sklearn.metrics import accuracy_score

In [28]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Isabella\OneDrive - Insper - Institudo de Ensino e Pesquisa\Documentos\Insper\2º SEMESTRE\CDADOS\22-1a-cd-p1-grupo_lucas-quadros_nicolas


Carregando a base de dados com as notícias classificadas manualmente:

In [29]:
filename = 'dados.xlsx'

In [30]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head()

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Corretora cripto,Binance suspende saques temporariamente devido...,"Nesta segunda-feira (1º), a corretora cripto B...",01/11/2021 - 15:26,77,2
1,Finanças Descentralizadas (DeFi),Protocolos DeFi representam uma indústria de U...,"De acordo com site rastreador DefiLlama, há, a...",13/10/2021 - 12:51,90,1
2,Mercados,Criptomoedas Terra (LUNA) e Avalanche (AVAX) l...,"Conforme noticiado pelo Decrypt, algumas cript...",23/02/2022 - 8:36,12,1
3,Finanças Descentralizadas (DeFi),Solana tem nova máxima histórica e ultrapassa ...,"Segundo o Decrypt, dados da CoinGecko mostram ...",25/10/2021 - 12:27,82,1
4,Blockchain,Capacidade da Lightning Network atinge novo re...,A capacidade da Lightning Network do Bitcoin a...,29/09/2021 - 16:48,98,1


In [31]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target
0,Corretora cripto,CEO da Crypto.com confirma que 400 contas fora...,"O CEO da Crypto.com, Kris Marszalek, confirmou...",19/01/2022 - 16:08,32,2
1,Tradicional vs. cripto,Gigante de investimentos Pimco poderá negociar...,"Pimco, empresa de investimentos que gerencia U...",21/10/2021 - 11:14,85,1
2,Investindo em cripto,Traders Club celebra contrato de US$ 15 milhõe...,"Na manhã desta quarta-feira (6), o Traders Clu...",06/10/2021 - 13:56,94,1
3,Investimentos,Comissão de Valores Mobiliários dos Estados Un...,A Comissão de Valores Mobiliários dos Estados ...,15/10/2021 - 19:00,88,1
4,Opinião,Caio Mesquita: Novo uniforme para uma longa ba...,"Desde o início, no agora longínquo 2009, temos...",18/12/2021 - 12:00,47,0


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

O assunto escolhido pelo grupo foi 'Criptomoedas', qualquer forma de moeda que existe digital ou virtualmente e usa criptografia para garantir a realização de transações. Foram escolhidas 3 variáveis para a target, sendo elas: as notícias neutras, as notícias boas e as notícias ruins para o mundo dos criptoativos. Dessa forma, para a classificação, o critério principal foi o impacto de cada notícia para as criptomoedas, assim, se uma notícia impactava positivamente, ela era classificada como boa (1); se ela impactava negativamente, era classificada como ruim (2) e se não causava impacto ou não estava relacionada, como neutra (0). Por exemplo: uma notícia sobre a queda ou desvalorização de uma moeda foi classificada como ruim; já sobre a valorização ou o aumento da procura por determinada cripto, foi classificada como boa; já uma notícia do tipo 'Conheça mais sobre as criptomoedas', foi classificada como neutra.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [32]:
def cleanup(text):
    """
        Função de limpeza que troca alguns sinais básicos e a string de célula vazia por espaços
    """
    #import string
    punctuation = '[!-.:?;''""]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    pattern2 = re.compile('\xa0')
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(pattern2, '', text_subbed)
    text_final = text_subbed.lower()
    
    return text_final


def stopwords(text):
    """
        Função de limpeza que remove preposições e artigos
    """
    prepositions = ['a', 'e', 'de', 'da', 'do', 'para', 'o', 'em', 'que', 'com', 'as', 'os', 'no', 'um', 'uma', 'é', 'por', 'seu', 'sua', 'na', 'no', 'por', 'ao', 'para', 'dos', 'das']
    new_text = ''
    for word in text.split():
        if word not in prepositions: 
            new_text += ' ' + word 
        else: 
            new_text = new_text
    return new_text
        

In [33]:
''' Transforma o conteúdo das colunas Titulo e PrimeiroParag em string
'''

train.Titulo = train.Titulo.astype('str')
train.PrimeiroParag = train.PrimeiroParag.astype('str')

In [34]:
''' Aplicação da limpeza
'''

train.PrimeiroParag = train.PrimeiroParag.apply(cleanup)
train.Titulo = train.Titulo.apply(cleanup)

train.PrimeiroParag = train.PrimeiroParag.apply(stopwords)
train.Titulo = train.Titulo.apply(stopwords)

In [35]:
''' Junção do Titulo e PrimeiroParag na nova coluna 'texto'
'''

for i in range(0,600):
    
    noticia = train['Titulo'][i] + ' ' + train['PrimeiroParag'][i] 
    
    train.loc[train['Titulo'] == train['Titulo'][i], 'texto'] = noticia 


In [36]:
''' Criação de grandes strings com todas as palavras contidas em cada variável 
'''

neutro = ''
bom = ''
ruim = ''

i = 0

for classe in train['Target']: 
    
    if classe == 0:
        
        neutro += ' ' + train['texto'][i]
        
    if classe == 1:
        
        bom += ' ' + train['texto'][i]
        
    if classe == 2:
        
        ruim += ' ' + train['texto'][i]
        
    i += 1
    

In [37]:
''' Split das grandes strings
'''
todas_palavras_neutro = neutro.split()
todas_palavras_bom = bom.split()
todas_palavras_ruim = ruim.split()

''' Série em value counts absoluta e relativa das palavras em cada categoria
'''
serie_neutra = pd.Series(todas_palavras_neutro).value_counts(True)
serie_boa = pd.Series(todas_palavras_bom).value_counts(True)
serie_ruim = pd.Series(todas_palavras_ruim).value_counts(True)

serie_neutra_abs = pd.Series(todas_palavras_neutro).value_counts()
serie_boa_abs = pd.Series(todas_palavras_bom).value_counts()
serie_ruim_abs = pd.Series(todas_palavras_ruim).value_counts()

'''Junção de todas as palvras e aplicação do Value Counts para excluir repetições
'''

todas_as_palavras = todas_palavras_neutro + todas_palavras_bom + todas_palavras_ruim

todas_as_palavras = pd.Series(todas_as_palavras).value_counts(True)


In [38]:
'''Cálculo das probabilidas de cada variável
'''
prob0 = len(todas_palavras_neutro)/ (len(todas_palavras_neutro)+len(todas_palavras_bom)+len(todas_palavras_ruim))
prob1 = len(todas_palavras_bom)/ (len(todas_palavras_neutro)+len(todas_palavras_bom)+len(todas_palavras_ruim))
prob2 = len(todas_palavras_ruim)/ (len(todas_palavras_neutro)+len(todas_palavras_bom)+len(todas_palavras_ruim))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [39]:
''' Transforma o conteúdo das colunas Titulo e PrimeiroParag em string
'''
test.Titulo = test.Titulo.astype('str')
test.PrimeiroParag = test.PrimeiroParag.astype('str')

In [40]:
''' Aplicação da limpeza
'''
test.PrimeiroParag = test.PrimeiroParag.apply(cleanup)
test.Titulo = test.Titulo.apply(cleanup)

test.PrimeiroParag = test.PrimeiroParag.apply(stopwords)
test.Titulo = test.Titulo.apply(stopwords)



In [41]:
''' Junção do Titulo e PrimeiroParag na nova coluna 'texto'
'''

for i in range(0,len(test)): 
    
    noticia = test['Titulo'][i] + ' ' + test['PrimeiroParag'][i] 
    
    test.loc[test['Titulo'] == test['Titulo'][i], 'texto'] = noticia 
    
test.texto = test.texto.astype('str')
    

In [42]:
'''Probabilidade texto dado neutro - com suavização de LaPlace
'''
lista_prob_texto_dado_0 = []

for frase in test['texto']:
    
    prob_texto_dado_0 = 1
    
    for word in frase.split():
        
        if word not in serie_neutra:
            
            prob_texto_dado_0 *= 1 / (sum(serie_neutra_abs) + len(todas_as_palavras))
            
        else:
            
            prob_texto_dado_0 *= (serie_neutra_abs[word] + 1) / (sum(serie_neutra_abs) + len(todas_as_palavras))
            
    lista_prob_texto_dado_0.append(prob_texto_dado_0)
    


In [43]:
'''Probabilidade texto dado bom - com suavização de LaPlace
'''
lista_prob_texto_dado_1 = []

for frase in test['texto']:
    
    prob_texto_dado_1 = 1
    
    for word in frase.split():
        
        if word not in serie_boa:
            
            prob_texto_dado_1 *= 1 / (sum(serie_boa_abs) + len(todas_as_palavras))
            
        else:
            
            prob_texto_dado_1 *= (serie_boa_abs[word] + 1) / (sum(serie_boa_abs) + len(todas_as_palavras))
                
    lista_prob_texto_dado_1.append(prob_texto_dado_1)
    


In [44]:
'''Probabilidade texto dado ruim - com suavização de LaPlace
'''
lista_prob_texto_dado_2 = []

for frase in test['texto']:
    
    prob_texto_dado_2 = 1
    
    for word in frase.split():
        
        if word not in serie_ruim: 
            
            prob_texto_dado_2 *= 1 / (sum(serie_ruim_abs) + len(todas_as_palavras))
            
        else:
            
            prob_texto_dado_2 *= (serie_ruim_abs[word] + 1) / (sum(serie_ruim_abs) + len(todas_as_palavras))
                
    lista_prob_texto_dado_2.append(prob_texto_dado_2)
    

In [45]:
'''Aplicação final do Nave Bayes e criação da coluna Classificador com o resultado 
'''
i = 0
for text in test['texto']:
    
    prob0dadoTexto = lista_prob_texto_dado_0[i] * prob0 
    prob1dadoTexto = lista_prob_texto_dado_1[i] * prob1 
    prob2dadoTexto = lista_prob_texto_dado_2[i] * prob2 
    
    
    if (prob0dadoTexto > prob1dadoTexto) and (prob0dadoTexto > prob2dadoTexto): 
        
        test.loc[test['Titulo'] == test['Titulo'][i], 'Classificador'] = 0
        
    elif (prob1dadoTexto > prob0dadoTexto) and (prob1dadoTexto > prob2dadoTexto): 
        
        test.loc[test['Titulo'] == test['Titulo'][i], 'Classificador'] = 1
        
    elif (prob2dadoTexto > prob1dadoTexto) and (prob2dadoTexto > prob0dadoTexto): 
        
        test.loc[test['Titulo'] == test['Titulo'][i], 'Classificador'] = 2
        
    i += 1

test.Classificador = test.Classificador.astype('int')
test.head()

,Categoria,Titulo,PrimeiroParag,Data,Pagina,Target,texto,Classificador
0,Corretora cripto,ceo cryptocom confirma 400 contas foram compr...,ceo cryptocom kris marszalek confirmou corret...,19/01/2022 - 16:08,32,2,ceo cryptocom confirma 400 contas foram compr...,2
1,Tradicional vs. cripto,gigante investimentos pimco poderá negociar c...,pimco empresa investimentos gerencia us 2 tri...,21/10/2021 - 11:14,85,1,gigante investimentos pimco poderá negociar c...,1
2,Investindo em cripto,traders club celebra contrato us 15 milhões 2...,manhã desta quartafeira 6 traders club inform...,06/10/2021 - 13:56,94,1,traders club celebra contrato us 15 milhões 2...,2
3,Investimentos,comissão valores mobiliários estados unidos a...,comissão valores mobiliários estados unidos s...,15/10/2021 - 19:00,88,1,comissão valores mobiliários estados unidos a...,2
4,Opinião,caio mesquita novo uniforme longa batalha,desde início agora longínquo 2009 temos dito ...,18/12/2021 - 12:00,47,0,caio mesquita novo uniforme longa batalha de...,0


In [46]:
'''Verificando a performance
'''
comparador = pd.crosstab(test['Target'], test['Classificador'],normalize = 'index')

falso_neutro = comparador[1][0]+comparador[2][0]
falso_positivo = comparador[0][1]+comparador[2][1]
falso_negativo = comparador[0][2]+comparador[1][2]

acuracia = accuracy_score(test['Target'], test['Classificador'])

In [47]:
print('Verdadeiros neutros: {:.2f}% \nFalsos neutros: {:.2f}%\nVerdadeiros positivos: {:.2f}% \nFalsos positivos: {:.2f}%\nVerdadeiros negativos: {:.2f}% \nFalsos negativos: {:.2f}%\nAcurácia: {:.2f}%'.format(comparador[0][0]*100, falso_neutro*100, comparador[1][1]*100, falso_positivo*100, comparador[2][2]*100, falso_negativo*100, acuracia*100))
        

Verdadeiros neutros: 68.80% 
Falsos neutros: 31.20%
Verdadeiros positivos: 62.01% 
Falsos positivos: 37.99%
Verdadeiros negativos: 56.25% 
Falsos negativos: 43.75%
Acurácia: 62.75%


___
### Concluindo

Por meio do algoritmo de Naive-Bayes, com a suavização de LaPlace aplicada, o grupo obteve um classificador com uma acurácia -método que indica a performance geral do modelo - de 62.75%, sendo esse um resultado satisfatório para essa etapa do curso de Ciência dos Dados, pois representa que mais da metade dos resultados do Classificador estavam em concordância com a marcação manual feita pelo grupo.  

Ademais, o classificador obteve diferentes performances para cada categoria: A categoria Neutra obteve o melhor desempenho individual, apresentando quase 70% de Verdadeiros-Neutros; Já a categora Negativa obteve o desempenho menos satisfatório entre esses, com 56,25% de Verdadeiros-Negativos. No entanto, esse valor é apenas 6,5% abaixo do maior desempenho individual nas variáveis, o que explicita equilíbrio entre elas.

Parte do desempenho individual ter sido um pouco mais baixo na categoria 'Negativa' pode ser atrelado à dificuldade do algoritmo em interpretar sarcasmos e dupla negação em um texto, uma vez que ele analisa cada palavra individualmente em uma frase, e não o contexto ou a ordem em que ela foi escrita. Por exemplo, uma frase do tipo: 'O bitcoin não aumentou hoje, depois de vários dias em alta' seria considerada uma notícia ruim para a classificação manual, no entanto, o Classificador provavelmente a interpretaria como uma notícia boa por conter, em sua maioria, palavras normalmente ligadas à categoria Positiva (aumentou, alta). 

Entretanto, esse aspecto que enviesa o Naive-Bayes pode ser melhorado por meio da combinação de atributos relacionados, através de algoritmos de regressão auxiliares, chamados Combinadores. Inicialmente, para selecionar as combinações possíveis, o algoritmo constrói um modelo tratando todos os aributos como condicionalmente independentes. Então, ele substitui cada par de atributos por um novo atributo combinado. A combinação que obter a melhor performance é mantida.  [https://www.cos.ufrj.br/~ines/enia07_html/pdf/28095.pdf]

Além disso, é preciso ter em mente que o Classificador atual está em uma de suas formas mais puras, podendo ser, assim, melhorado e desenvolvido posteriormente e, por isso, o projeto deve continuar sendo financiado. Nesse sentido, ele pode expandir e abrangir ainda mais o campo das Criptomoedas: além de classificá-las, pode indicar a qual Cripto específica aquela notícia se refere e, assim, fazer um estudo mais profundo do mundo cripto como um todo. 

Decerto, o modelo Naive-Bayes é um dos mais populares no aprendizado de máquinas, sendo assim, possui outras aplicações além da classificação de notícias. Entre elas, há o 'Sistema de Recomendação': analisar indivíduos e sugerir algo que possa os interessar. No Naive-Bayes, a filtragem colaborativa seria o método utilizado para esse tipo de aplicação,o que consistiria na percepção de similaridades entre usuários e determinação de suas preferências. Outro uso é o sistema utilizado na saúde, para prever a disposição de determinada doença em um usuário: análise de dados históricos e condições correlatas entre diferentes tipos de doenças - por exemplo, por meio dos sintomas e características de um determinado paciente que possui uma doença A decobrir qual as chances de ele desenvolver uma outra doença B. 

O último aspecto a ser considerado nessa discussão é o fato de que não se pode usar o próprio Classificador para gerar novas amostras de treinamento, pois isso seria o equivalente a comparar um 'modelo' com outro 'modelo'. Um exemplo prático e cotidiano seria fazer uma lista de exercícios para treinar para uma determinada prova e depois receber a prova idêntica à lista que foi usada de treinamento, seria o mesmo que decorar as respostas. Assim seria com o Classificador, ele não estaria, de fato, criando novas amostras, apenas replicando sua própria base. 

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**